In [ ]:
from fastai.vision.all import *
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
import pandas as pd
import os
import random
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv', header=0, names=['id','value'], dtype=object)
df = df[~df.id.isin(["00109", "00123", "00709"])]

In [ ]:
df.head()

In [ ]:
#https://stackoverflow.com/a/4836734/8245487
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [ ]:
import os
import pydicom
import pandas as pd
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm
import binascii
from PIL import Image

INPUT = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'

if not os.path.exists('./train'):
    os.makedirs('./train')
    

if not os.path.exists('./test'):
    os.makedirs('./test')


def get_dicom_files(input_dir, dataset='train'):
    for subdir, dirs, files in os.walk(f"{input_dir}/{dataset}"):
        if len(files) == 0:
            continue
        filename = natural_sort(files)[len(files)//2] #take middle most image -- FLAIR DCM file per training item.
        filepath = os.path.join(subdir, filename)
        
        if filepath.endswith(".dcm") and "FLAIR" in filepath:
            cur_id = subdir.split('/')[-2]
            outpath = os.path.join(f'./{dataset}',f'{cur_id}.png')
            
            process_dicom(filepath, outpath)

def process_dicom(path, outpath):
    dicom = pydicom.read_file(path)
    data = apply_voi_lut(dicom.pixel_array, dicom)
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    height = len(data)
    width = len(data[0])
    
    pixels_out = []
    for row in data:
        pixels_out.extend(row)
    assert(len(pixels_out) == height * width)
    
    image_out = Image.new('L', (width, height))
    image_out.putdata(pixels_out)
    image_out.save(outpath)

get_dicom_files(INPUT, 'train')
get_dicom_files(INPUT, 'test')


#     final = pd.DataFrame(final)
#     final.to_csv(f"{args['output']}/dicom_meta_{args['dataset']}.csv", index=False)

In [ ]:
for id_num in df.id:
    full_path = './train/{}.png'.format(id_num)
    df.loc[df.id == id_num, 'file'] = full_path
    

In [ ]:
df

In [ ]:
dls = ImageDataLoaders.from_df(df, item_tfms=Resize(224), bs=64, label_col =1, fn_col=2, path='')

In [ ]:
dls.show_batch()

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
print(Net())

In [ ]:

learn = cnn_learner(dls, Net, metrics=[error_rate, accuracy], model_dir="/tmp/model/").to_fp16()

In [ ]:
learn.loss_func

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, lr_max=1e-2)

In [ ]:
learn.show_results()

In [ ]:
df_test = pd.DataFrame(columns=['id', 'value'])
df_test.id = os.listdir("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/")

In [ ]:
for id_num in df_test.id:
    full_path = './test/{}.png'.format(id_num)
    prediction = learn.predict(full_path)
    print(prediction)
    probability = prediction[2][1].item()
    print(probability)
    df_test.loc[df_test.id==id_num, 'value'] = probability

In [ ]:
df_test.head()

In [ ]:
df_test.value.min()

In [ ]:
df_test.value.max()

In [ ]:
df_test.rename(columns={'id':'BraTS21ID','value':'MGMT_value'}).to_csv('submission.csv', index=False)